In [ ]:
import os
import math
import glob
import pymysql
import pandas as pd
from sqlalchemy import create_engine
from tqdm.notebook import tqdm


from django.conf import settings
from appcreator import creator
from appcreator.import_utils import fetch_models, import_and_create_m2m, import_m2m_tables

In [ ]:
dbc = settings.LEGACY_DB_CONNECTION
sheet_id = "1dd9OlPFYCK1bHdC8U1eax13glEitlZhTDc-Cri_qA5A"

In [ ]:
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [ ]:
query = f"SELECT * FROM verpackung"

In [ ]:
df = pd.read_sql(query, con=db_connection)

In [ ]:
scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='gebinde'
)
col, _ = SkosCollection.objects.get_or_create(
    name='gebinde'
)
for i, row in df.iterrows():
    leg_id = f"gebinde__{float(row['verpackungId'])}"
    try:
        item, _ = SkosConcept.objects.get_or_create(legacy_id=leg_id)
    except:
        continue
    item.pref_label = f"{row['verpackung']}"
    item.scheme.add(scheme)
    item.collection.add(col)
    item.save()

In [ ]:
query = f"SELECT * FROM einheiten"

In [ ]:
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in df.iterrows():
    leg_id = f"einheit__{float(row['einheitId'])}"
    try:
        item = SkosConcept.objects.get(legacy_id=leg_id)
    except:
        continue
    item.pref_label = f"{row['einheit']}"
    item.save()

In [ ]:
query = f"SELECT * FROM fahrtrichtung"
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in df.iterrows():
    leg_id = f"fahrtrichtung__{float(row['fahrtrichtungId'])}"
    try:
        item = SkosConcept.objects.get(legacy_id=leg_id)
    except:
        continue
    item.pref_label = f"{row['fahrtrichtung']}"
    item.save()

In [ ]:
query = f"SELECT * FROM fahrzeuge_anzahlTyp"
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        fz = Fahrzeug.objects.get(legacy_pk=row['fahrzeugId'])
    except:
        continue
    try:
        concept, _ = SkosConcept.objects.get_or_create(
            legacy_id=f"schiff_typ__{float(row['fahrzeugTyp'])}"
        )
    except:
        continue
    anzahl = f"{row['fahrzeugAnzahl']}"
    if fz is not None:
        item, _ = SchiffTyp.objects.get_or_create(
            id=row['fahrzeug_anzahlTypId']
        )
        item.legacy_pk = row['fahrzeug_anzahlTypId']
        item.legacy_id = float(row['fahrzeug_anzahlTypId'])
        item.fahrzeug = fz
        item.skosconcept = concept
        item.anzahl = anzahl
        item.save()

In [ ]:
query = f"SELECT * FROM fahrzeugTypen"
df = pd.read_sql(query, con=db_connection)

In [ ]:
scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='schiff_typ')
col, _ = SkosCollection.objects.get_or_create(name='schiff_typ')
for i, row in df.iterrows():
    leg_id = f"schiff_typ__{float(row['fahrzeugTypId'])}"
    try:
        item = SkosConcept.objects.get(legacy_id=leg_id)
    except:
        continue
    item.pref_label = f"{row['fahrzeugTyp']}"
    item.scheme.add(scheme)
    item.collection.add(col)
    item.save()

In [ ]:
props = {
    'fahrzeuge_anzahlTyp_leer': 'leer',
    'fahrzeuge_anzahlTyp_mitGeschirr': 'geschirr',
    'fahrzeuge_anzahlTyp_mitRossen': 'rossen',
    'fahrzeuge_anzahlTyp_mitSG': 'mit_sg',
    'fahrzeuge_anzahlTyp_mitSichSelbst': 'mit_sich_selbst'
}

In [ ]:
for table, prop in tqdm(props.items(), total=len(props)):
    query = f"SELECT * FROM {table}"
    df = pd.read_sql(query, con=db_connection)
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:
            item = SchiffTyp.objects.get(legacy_pk=row[0])
        except:
            continue
        setattr(item, prop, row[1])
        item.save()

In [ ]:
query = f"SELECT * FROM ladung_mengeVerpackung"

In [ ]:
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    item, _ = MengeGebinde.objects.get_or_create(
        legacy_pk=row['ladung_mengeVerpackungId']
    )
    item.legacy_id = float(row['ladung_mengeVerpackungId'])
    try:
        ladung = Ladung.objects.get(legacy_pk=row['ladungId'])
        item.ladung = ladung
    except:
        pass
    item.menge = f"{row['menge']}"
    try:
        concept = SkosConcept.objects.get(legacy_id=f"gebinde__{float(row['gebinde'])}")
        item.gebinde = concept
    except:
        pass
    item.save()
    ladung.menge_gebinde.add(item)

In [ ]:
# if the previous cell worked, executing this cell should not be necessary
# for i, row in tqdm(df.iterrows(), total=df.shape[0]):
#     try:
#         ladung = Ladung.objects.get(legacy_pk=row['ladungId'])
#     except:
#         continue
#     item = MengeGebinde.objects.get(
#         legacy_pk=row['ladung_mengeVerpackungId']
#     )
#     ladung.menge_gebinde.add(item)

In [ ]:
ladung.menge_gebinde.all()

In [ ]:
query = f"SELECT * FROM personen_herkunft"
df = pd.read_sql(query, con=db_connection)

In [ ]:
for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        person = Person.objects.get(legacy_pk=row['personenId'])
    except:
        continue
    try:
        place = Ort.objects.get(legacy_pk=row['herkunft'])
    except:
        continue
    person.herkunft = place
    person.save()